# Aula3.Ex5 - Quadrados

### Primeiro, importamos as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np


### Inicializando janela

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
window = glfw.create_window(720, 600, "Quadrados", None, None)
glfw.make_context_current(window)


### Capturando eventos de teclado e mouse

In [3]:
posicaoX = 0.0
posicaoY = 0.0

def mouse_event(window, button, action, mods):

    print('-------')
    x, y = glfw.get_cursor_pos(window)
    posicaoX = x / 720
    posicaoY = y / 600
    # print(posicaoX)


glfw.set_mouse_button_callback(window, mouse_event)


### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec2.
* Usamos vec2, pois nosso programa (na CPU) irá enviar apenas duas coordenadas para plotar um ponto. Podemos mandar três coordenadas (vec3) e até mesmo quatro coordenadas (vec4).
* void main() é o ponto de entrada do nosso programa (função principal).
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4.

In [4]:
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transform;
        void main(){
            gl_Position = mat_transform * vec4(position,0.0,1.0);
        }
        """


### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal).
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

In [5]:
fragment_code = """
        void main(){
            gl_FragColor = vec4(0.0, 0.0, 0.0, 1.0);
        }
        """


### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [6]:
# Request a program and shader slots from GPU
program = glCreateProgram()
vertex = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [7]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)


### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [8]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [9]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")


### Associando os programas compilado ao programa principal

In [10]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [11]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')

# Make program the default program
glUseProgram(program)


### Preparando dados para enviar a GPU

Nesse momento, compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


In [12]:
# preparando espaço para 4 vértices usando 2 coordenadas (x,y)
vertices = np.zeros(4, [("position", np.float32, 2)])


In [13]:
# preenchendo as coordenadas de cada vértice
vertices['position'] = [
    (+0.2, +0.4),
    (+0.3, +0.4),
    (+0.2, +0.5),
    (+0.3, +0.5)
]


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [14]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)


### Abaixo, enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [15]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)


### Associando variáveis do programa GLSL (Vertex Shaders) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [16]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Definimos essa variável no Vertex Shader.

In [17]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)


A partir da localização anterior, indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [18]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)


### Nesse momento, exibimos a janela.


In [19]:
glfw.show_window(window)


### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

A novidade agora é a função glDrawArrays().

Tal função recebe o tipo de primitiva (GL_TRIANGLES), o índice inicial do array de vértices (vamos mostrar todos os três vértices, por isso começamos com 0) e a quantidade de vértices ( len(vertices) ).

In [20]:
import math
d = 0.0
sentidoTransY = False
sentidoTrans = False
sentidoGiro = False
x = 0.25
y = 0.45
diferenca = 0.09


def multiplica_matriz(a, b):
    m_a = a.reshape(4, 4)
    m_b = b.reshape(4, 4)
    m_c = np.dot(m_a, m_b)
    c = m_c.reshape(1, 16)
    return c

lastPx = 1


while not glfw.window_should_close(window):

    glfw.poll_events()

    # if(posicaoX <= -1.2):
    #     sentidoTrans = True
    # elif(posicaoX >= 0.7):
    #     sentidoTrans = False

    # if(sentidoTrans):
    #     posicaoX += 0.01
    # else:
    #     posicaoX -= 0.01

    # if(posicaoY <= -0.3):
    #     sentidoTransY = True
    # elif(posicaoY >= 0.2):
    #     sentidoTransY = False


    # if(sentidoTransY):
    #     posicaoY += 0.003
    # else:
    #     posicaoY -= 0.003


    if(lastPx != posicaoX):
        print(posicaoX)
        lastPx = posicaoX

    d -= diferenca

    cos_d = math.cos(d)
    sen_d = math.sin(d)

    mat_transla = np.array([1.0, 0.0, 0.0, posicaoX,
                            0.0, 1.0, 0.0, posicaoY,
                            0.0, 0.0, 1.0, 0.0,
                            0.0, 0.0, 0.0, 1.0
                            ], np.float32)

    mat_translate = np.array([1.0, 0.0, 0.0, x,
                             0.0, 1.0, 0.0, y,
                             0.0, 0.0, 1.0, 0.0,
                             0.0, 0.0, 0.0, 1.0
                              ], np.float32)

    mat_rotation = np.array([cos_d, -sen_d, 0.0, 0.0,
                             sen_d, cos_d, 0.0, 0.0,
                             0.0, 0.0, 1.0, 0.0,
                             0.0, 0.0, 0.0, 1.0
                             ], np.float32)

    mat_minus_translate = np.array([1.0, 0.0, 0.0, -x,
                                    0.0, 1.0, 0.0, -y,
                                    0.0, 0.0, 1.0, 0.0,
                                    0.0, 0.0, 0.0, 1.0
                                    ], np.float32)


    mat_transformation = multiplica_matriz(mat_transla, mat_translate)
    mat_transformation = multiplica_matriz(mat_transformation, mat_rotation)
    mat_transformation = multiplica_matriz(
        mat_transformation, mat_minus_translate)

    loc = glGetUniformLocation(program, "mat_transform")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)

    # glPolygonMode(GL_FRONT_AND_BACK,GL_LINE) ## ative esse comando para enxergar os triângulos
    glClear(GL_COLOR_BUFFER_BIT)
    glClearColor(1.0, 1.0, 1.0, 1.0)

    glDrawArrays(GL_TRIANGLE_STRIP, 0, 4)

    glfw.swap_buffers(window)

glfw.terminate()


0.0
-------
-------
-------
-------


# Exercício

Modifique esse código para desenhar retângulos.
